In [6]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Lambda, Dropout
from sklearn.model_selection import train_test_split
from keras import backend as K

The path part needs to be changed according to where you keep the data set and what OS you use.

In [7]:
path = '/Users/maksymb/Desktop/work/Courses/FYS-STK4155-Applied_data_analysis_and_machine_learning/machine-learning-projects/p3/data/'#"e:\\Data\\Skole\\Project3\\"
data = pd.read_csv("{}ASV_table.tsv".format(path))
meta = pd.read_csv("{}Metadata_table.tsv".format(path),delimiter=r"\s+")

In [8]:
def normalize(x):
    return (x-np.amin(x))/(np.amax(x)-np.amin(x))

I think this part might be one of the more important in regards to what sorts of results can be achieved. Choosing an apropriate way to categorize the data points is essential and not, to me, obvious. The though behind the make_categories function was to classify the lakes into, say, high/low pH. It takes a target variable vector, such as lake pH, and a number N, and splits the range of values into N categories.

For good results I think the categories should be set manually according to biologically more relevent cutoffs. This function just splits the range of values evenly.

In [9]:
def make_categories(data,number_of_categories):
    '''
    arguments: data , vector containing continous data to be separated int
    categories i.e. high, med, low
    numbeer_of_categories, determines how many categories the data
    set is split into
    
    returns 
    '''
    #Create "size" of each category. 
    boundries = (data.max()-data.min())/number_of_categories 
    #Create equaly sized bins for each category
    cat = [data.min()+(1+i)*boundries for i in range(number_of_categories-1)]
    categorized_data = np.digitize(data,cat)        
    return categorized_data

This function is "the heart" of the oneshot. For each data point, It pairs it with one data point of the same class and one data point of a different class. The neural network will then use these pairs to learn to tell if to points "look alike". 

In [10]:
def make_pairs(data,target):
    '''
    This function returns pairs of data points for comparison
    in siamese oneshot network
    '''
    #Calculates number of classe in an array where classes
    #are idexed as 0,1,2...,n
    num_classes = target.max()+1
    #Find the indices for each representative of each class
    #and put them in a list
    class_index = [np.where(target==i)[0] for i in range(num_classes)]
    
    #create lsits to store pairs and labels to return
    pairs = []
    labels = []
    
    #Loop through data points and make matching and non matching pairs
    for index in range(len(data)):
        #pick a data point
        x1 = data[index]
        #Get label of data point
        label1 = target[index]
        #Pick an index corresponding to a data point of matching label
        random_index = np.random.choice(class_index[label1])
        x2 = data[random_index]
        #Store pairs and label it a match (1 is match 0 is miss)
        pairs += [[x1,x2]]
        labels += [1]
        
        #Find a non matching data point, first random draw
        classes_represented = np.unique(target)
        label2 = np.random.choice(classes_represented)
        #Check that the labels are different
        while label1 == label2 and num_classes>1:
            #If not, then draw another sample
            label2 = np.random.choice(classes_represented)
        #Pick a representative from the differently labeled class
        random_index = np.random.choice(class_index[label2])
        x2 = data[random_index]
        
        #Store pairs and label it a match (1 is match 0 is miss)
        pairs += [[x1,x2]]
        labels += [0]
    
    return np.array(pairs),np.array(labels)

This function calculates the distance between vectors and is how the network measures similarity.

In [11]:
def euclidean_distance(vectors):
    x, y = vectors
    sum_square = K.sum(K.square(x-y),axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

Below one can set what metadata to use as a target. Currently set to test N2O classification.

In [12]:
data = normalize(data)
ph = meta["pH"]
n2o = meta["N2O"]
#Some data formating        
y = make_categories(n2o,3)
X = data.to_numpy()

#Split into training and testing
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5)

#Make pairs and labels
pairs_train, labels_train = make_pairs(X_train,y_train)
pairs_test, labels_test = make_pairs(X_test,y_test)
#Set input size for network
input_size = len(X[0,:])

TypeError: unsupported operand type(s) for -: 'str' and 'str'

The next cell sets up the underlying network. Here you can change activation functions, number of layers, number of nodes, types of layers etc

In [ ]:
def base_network(input_shape):
    base_input = Input(shape=input_shape)
    x = Dense(100,activation="relu")(base_input)
    x = Dropout(0.1)(x)
    x = Dense(100,activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(100,activation="relu")(x)
    return Model(base_input,x)

A couple of helper functions. The first sets the output shape for the network. The second just calculates number of correctly classified data points

In [ ]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def results(data,target,cutoff=0.5):
    total = 0
    hits = 0
    for d,t in zip(data,target):
        if d < cutoff:
            if t == 1:
                total += 1
                hits += 1
            else:
                total += 1
        else:
            if t == 1:
                total += 1
            else:
                total +=1
                hits += 1
    return hits/total
    
def contrastive_loss(y_true, y_pred):

    '''Contrastive loss from Hadsell-et-al.'06

    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf

    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square) 

The next part finalizes the siamese network. It allows one to feed 2 inputs through the same network and compare results. It also sets loss functions etc. 

In [ ]:
base_model = base_network((input_size,))

input1 = Input(shape=(input_size,))
input2 = Input(shape=(input_size,))

run1 = base_model(input1)
run2 = base_model(input2)

merge_layer = Lambda(euclidean_distance,output_shape=eucl_dist_output_shape)(
        [run1,run2])
out_layer = Dense(1,activation="sigmoid")(merge_layer)
siamese_model = Model(inputs=[input1,input2],outputs=out_layer)

siamese_model.compile(loss=contrastive_loss,
                optimizer="RMSprop",metrics=["accuracy"])

siamese_model.summary()

This part trains and tests the model.

In [ ]:
siamese_model.fit([pairs_train[:,0],pairs_train[:,1]],
            labels_train[:],batch_size=5,epochs=40)

pred = siamese_model.predict([pairs_train[:,0],pairs_train[:,1]])
print("Got {} percent correct on training".format(results(pred,labels_train)*100))
pred = siamese_model.predict([pairs_test[:,0],pairs_test[:,1]])
print("Got {} percent correct on test".format(results(pred,labels_test)*100))